# Cell type classification

In [ ]:
# Import Packages

%load_ext autoreload
%autoreload 2

import os
import warnings 
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sns
import matplotlib.pyplot as plt
# from skimage.filters import threshold_otsu, gaussian
# from skimage.morphology import remove_small_objects
from matplotlib.colors import ListedColormap
from anndata import AnnData, concat

# Customized packages
from starmap.utilities import *
# from starmap.sequencing import *
# from starmap.obj import STARMapDataset, load_data
# import starmap.analyze as anz
# import starmap.viz as viz
import starmap.sc_util as su

# test()

In [ ]:
# Get functions 

import colorsys
from random import shuffle

def intervals(parts, start_point, end_point):
    duration = end_point - start_point
    part_duration = duration / parts
    return [((i * part_duration + (i + 1) * part_duration)/2) + start_point for i in range(parts)]

## IO

In [ ]:
# Set path
base_path = 'Z:/Data/Analyzed/2021-11-23-Hu-MouseBrain/'
out_path = os.path.join(base_path, 'output')
fig_path = os.path.join(base_path, 'figures')

out_path = os.path.join(base_path, 'output')
if not os.path.exists(out_path):
    os.mkdir(out_path)
    
fig_path = os.path.join(base_path, 'figures')
if not os.path.exists(fig_path):
    os.mkdir(fig_path)

## Input

In [ ]:
adata = sc.read_h5ad(os.path.join(out_path, '2022-03-12-Hu-TissueRIBOmap-4mad-filtered.h5ad'))
adata

## Preprocessing

In [ ]:
# Normalization scaling
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)

adata.layers['norm'] = adata.X
adata.raw = adata

In [ ]:
adata.layers['norm'].min()

In [ ]:
# Scale data to unit variance and zero mean
sc.pp.scale(adata)
adata.layers['scaled'] = adata.X

# Batch correction
# sc.pp.combat(adata)
# adata.layers['corrected'] = adata.X

In [ ]:
adata

## Level_1 clustering

### clustering

In [ ]:
# Run PCA
sc.tl.pca(adata, svd_solver='arpack', use_highly_variable=True)

# Plot explained variance 
sc.pl.pca_variance_ratio(adata, log=False)

In [ ]:
# Plot PCA
sc.pl.pca(adata, color='protocol')

In [ ]:
%%time
# Computing the neighborhood graph
n_neighbors = 50
n_pcs = 30 ## 30
min_dist = 0.05

sc.pp.neighbors(adata, n_neighbors=n_neighbors, n_pcs=n_pcs)

# Run UMAP
sc.tl.umap(adata, min_dist=min_dist, random_state=0) ## 0.5

In [ ]:
%%time
# Run leiden cluster
cluster_resolution = 1
sc.tl.leiden(adata, resolution = cluster_resolution, random_state=0)

# Plot UMAP with cluster labels 
sc.pl.umap(adata, color='leiden')
n_clusters = adata.obs['leiden'].unique().shape[0]

# Save log
with open(f'{fig_path}/log.txt', 'w') as f:
    f.write(f"""Number of neighbor: {n_neighbors}
Number of PC: {n_pcs}
Resolution: {cluster_resolution}
Min-distance: {min_dist}
Number of clusters: {n_clusters}""")

In [ ]:
# Plot single meta UMAP
sc.pl.umap(adata, color='protocol')

In [ ]:
sc.pl.umap(adata, color='total_counts')
sc.pl.umap(adata, color='n_genes')

In [ ]:
# Get colormap
cluster_pl = sns.color_palette("hls", n_clusters)
cluster_cmap = ListedColormap(cluster_pl.as_hex())
sns.palplot(cluster_pl)

In [ ]:
# Get markers for each cluster
sc.tl.rank_genes_groups(adata, 'leiden', method='wilcoxon', pts=True)

In [ ]:
# Filter markers
sc.tl.filter_rank_genes_groups(adata, min_fold_change=1)

In [ ]:
current_leiden_group = '6'

current_df = sc.get.rank_genes_groups_df(adata, group=current_leiden_group, key='rank_genes_groups')
current_df.head(10)

In [ ]:
# Dot plot mean expression (##)
sc.pl.rank_genes_groups_dotplot(adata, key='rank_genes_groups_filtered', n_genes=5, dendrogram=False)

In [ ]:
# Other type of plots
# Plot z-score heatmap
sc.pl.rank_genes_groups_heatmap(adata, n_genes=5, groupby='leiden', min_logfoldchange=1, use_raw=False, swap_axes=True, 
                                vmin=-5, vmax=5, cmap='bwr', show_gene_labels=True,
                                dendrogram=False, figsize=(30, 20), save=False)

In [ ]:
sc.pl.rank_genes_groups_heatmap(adata, n_genes=5, groupby='leiden', min_logfoldchange=1, use_raw=False, swap_axes=True, 
                                vmin=-5, vmax=5, cmap='bwr', show_gene_labels=True, key='rank_genes_groups_filtered', 
                                dendrogram=False, figsize=(30, 20), save=False)

In [ ]:
gene_list = ['Slc17a7', 'Gad1', 'Gad2', 'Sst', 'Pvalb', 'Slc1a3', 'Aldoc', 'Bsg', 'Ctss', 'Plp1', 'Mobp', 'Pdgfra', 'Dcn', 'Vim', 'Gfap']

fig, axs = plt.subplots(nrows=3, ncols=5, figsize=(25, 12))
axs = axs.flatten()
for i, gene in enumerate(gene_list):
    ax = sc.pl.umap(adata, color=gene, title=gene, ax=axs[i], show=False)
    
plt.show()

### assign label

In [ ]:
# Print markers 
markers = []
temp = pd.DataFrame(adata.uns['rank_genes_groups']['names']).head(10)
for i in range(temp.shape[1]):
    curr_col = temp.iloc[:, i].to_list()
    markers = markers + curr_col
    # print(i, curr_col)
    print(i)
    for j in curr_col:
        print(j, end=',')
    print('')

In [ ]:
# Plot UMAP with cluster labels w/ new color
fig, ax = plt.subplots(figsize=(10,7))
sc.pl.umap(adata, color='leiden', legend_loc='on data',
           legend_fontsize=12, legend_fontoutline=2, frameon=False, 
           title='Top level clustering (leiden)', palette=cluster_pl, save=False, ax=ax)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
sns.scatterplot(x='column', y='row', hue='leiden', data=adata.obs, palette=cluster_pl, s=2, legend=False)
ax.axis('off')
plt.show()

In [ ]:
# Plot single cluster
fig, ax = plt.subplots(figsize=(10, 10))
sns.scatterplot(x='column', y='row', hue='leiden', data=adata.obs.loc[adata.obs['leiden'] == '10', :], palette='tab20', s=2, legend=False, ax=ax)
ax.axis('off')
plt.show()

In [ ]:
# Change cluster label to cell type label
transfer_dict = {}
level_1_list = ['Neuron', 'Neuron', 'Neuron', 'Vascular cells', 'Glia', 'Glia', 'Neuron', 'Glia', 'Neuron', 'Glia', 
                'Neuron', 'Neuron', 'Glia', 'Vascular cells', 'Neuron', 'Neuron', 'Neuron', 'Glia', 'Glia',
               'Neuron', 'Vascular cells']

for i in sorted(adata.obs['leiden'].unique()):
    transfer_dict[i] = level_1_list[int(i)]

In [ ]:
# Print markers 
markers = []
temp = pd.DataFrame(adata.uns['rank_genes_groups']['names']).head(10)
for i in range(temp.shape[1]):
    curr_col = temp.iloc[:, i].to_list()
    markers = markers + curr_col
    # print(i, curr_col)
    print(f"{i} - {level_1_list[i]}")
    for j in curr_col:
        print(j, end=',')
    print('')

In [ ]:
# Assign cell type to sdata
adata.obs['level_1'] = adata.obs['leiden'].values
adata.obs = adata.obs.replace({'level_1': transfer_dict})

# Remove NA 
adata = adata[adata.obs['level_1'] != 'NA', :]

# Sort category
level_1_order = ['Neuron', 'Glia', 'Vascular cells']

adata.obs['level_1'] = adata.obs['level_1'].astype('category')
adata.obs['level_1'].cat.reorder_categories(level_1_order, inplace=True)

In [ ]:
# Check color legend
level_1_order = ['Neuron', 'Glia']
adata.obs['level_1'] = adata.obs['level_1'].astype(object)
adata.obs['level_1'] = adata.obs['level_1'].astype('category')
adata.obs['level_1'] = adata.obs['level_1'].cat.reorder_categories(level_1_order)
level_1_pl = sns.color_palette("hls", 2)
sns.palplot(level_1_pl, size=3)
plt.xticks(range(len(level_1_order)), level_1_order, size=10, rotation=45)
plt.tight_layout()
# plt.savefig(f'./figures/color_legend_top.png')
plt.show()

In [ ]:
# Save plots
# Plot UMAP with cluster labels w/ new color
sc.pl.umap(adata, color='level_1', legend_loc='on data',
           legend_fontsize=12, legend_fontoutline=2, frameon=False, 
           title='Level 1 clustering', palette=level_1_pl, save=False)

# sc.tl.rank_genes_groups(adata, 'level_1', method='wilcoxon')

# # Plot z-score heatmap
# sc.pl.rank_genes_groups_heatmap(adata, n_genes=5, groupby='top_level', min_logfoldchange=1, use_raw=False, swap_axes=True, 
#                                 vmin=-5, vmax=5, cmap='bwr', show_gene_labels=True,
#                                 dendrogram=False, figsize=(30, 15), save=False)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
sns.scatterplot(x='column', y='row', hue='level_1', data=adata.obs, palette=level_1_pl, s=2, legend=False)
ax.axis('off')
plt.show()

In [ ]:
sc.pl.dotplot(adata, ['Slc17a7', 'Gad1', 'Gad2', 'Plp1', 'Mbp', 'Aldoc'], 'level_1', dendrogram=False)

In [ ]:
# backup 
from datetime import datetime
date = datetime.today().strftime('%Y-%m-%d')
adata.write_h5ad(f"{out_path}/{date}-Hu-TissueRIBOmap-level1-bk.h5ad")

### plot tile number

In [ ]:
# load coords_df
coords_df4 = pd.read_csv('Z:/jiahao/Github/RIBOmap/segmentation-stitching/RIBOmap/results/tuned_coords.csv', index_col=0)
coords_df4 = coords_df4.loc[coords_df4['tile'] != 0, :]
coords_df4

In [ ]:
# tile_col = coords_df4['column'].values + 1000
tile_row = coords_df4['row'].values + 1000

fig, ax = plt.subplots(figsize=(20,20))
plt.scatter(adata.obs['row'], adata.obs['column'], s=1, c='r')
for i in range(coords_df4.shape[0]):
    plt.text(tile_row[i], tile_col[i], s=coords_df4['tile'].astype(str).to_list()[i])
ax.set_aspect('equal')
plt.show()

## Level_2 clustering

In [ ]:
# Load backup dataset 
adata = sc.read_h5ad(os.path.join(out_path, '2022-03-16-Hu-TissueRIBOmap-level1-bk.h5ad'))
adata

In [ ]:
# Embedding parameters
emb_dict = {
    'Neuron': {'n_neighbors': 50, 'n_pcs': 10, 'min_dist': .1, 'cluster_resolution': 2},
    'Glia': {'n_neighbors': 50, 'n_pcs': 15, 'min_dist': .1, 'cluster_resolution': 1.2},
}

save_embedding = False

In [ ]:
# Subset
sub_id = 'Glia'
curr_cells = adata.obs['level_1'] == sub_id
sdata = adata[curr_cells, :]
sdata

In [ ]:
sub_level_fig_path = os.path.join(fig_path, sub_id)
if not os.path.exists(sub_level_fig_path):
    os.mkdir(sub_level_fig_path)

### clustering

In [ ]:
# Run PCA
sc.tl.pca(sdata, svd_solver='arpack', use_highly_variable=True)

# Plot explained variance 
sc.pl.pca_variance_ratio(sdata, log=False)

# Plot PCA
sc.pl.pca(sdata, color='protocol')

In [ ]:
%%time
# Computing the neighborhood graph
n_neighbors = emb_dict[sub_id]['n_neighbors']
n_pcs = emb_dict[sub_id]['n_pcs']
min_dist = emb_dict[sub_id]['min_dist']

sc.pp.neighbors(sdata, n_neighbors=n_neighbors, n_pcs=n_pcs, random_state=0)

# Run UMAP
sc.tl.umap(sdata, min_dist=min_dist, random_state=0)
sc.tl.diffmap(sdata, n_comps=n_pcs, random_state=0)

In [ ]:
%%time
# Run leiden cluster
cluster_resolution = emb_dict[sub_id]['cluster_resolution']
sc.tl.leiden(sdata, resolution = cluster_resolution)

# Plot UMAP with cluster labels 
sc.pl.umap(sdata, color='leiden')
sc.pl.diffmap(sdata, color='leiden')
n_clusters = sdata.obs['leiden'].unique().shape[0]

if save_embedding:
    # Save log
    with open(f'{fig_path}/log_{sub_id}.txt', 'w') as f:
        f.write(f"""Number of neighbor: {n_neighbors}
    Number of PC: {n_pcs}
    Resolution: {cluster_resolution}
    Min-distance: {min_dist}
    Number of clusters: {n_clusters}""")

    # save embeddings
    np.savetxt(f'{fig_path}/embedding_{sub_id}_umap.csv', sdata.obsm['X_umap'], delimiter=",")
    np.savetxt(f'{fig_path}/embedding_{sub_id}_diffmap.csv', sdata.obsm['X_diffmap'], delimiter=",")

In [ ]:
# sc.pl.dotplot(sdata, ['Slc17a7', 'Gad1', 'Gad2'], 'leiden', dendrogram=False)
sc.pl.dotplot(sdata, ['Plp1', 'Mbp', 'Gfap', 'Ctss'], 'leiden', dendrogram=False)

In [ ]:
# Get colormap
cluster_pl = sns.color_palette("hls", n_clusters)
cluster_cmap = ListedColormap(cluster_pl.as_hex())
sns.palplot(cluster_pl)

In [ ]:
# Add log layer
sdata.layers['log_raw'] = np.log1p(sdata.layers['raw'])
sc.pp.normalize_total(sdata, layer='log_raw')

# Find gene markers for each cluster
sc.tl.rank_genes_groups(sdata, 'leiden', method='wilcoxon', layer='log_raw', pts=True, use_raw=False, n_genes=adata.shape[1])

# Filter markers
sc.tl.filter_rank_genes_groups(sdata, min_fold_change=.1, min_in_group_fraction=0.2, max_out_group_fraction=0.8)

In [ ]:
current_cell_type = '15'

current_df = sc.get.rank_genes_groups_df(sdata, group=current_cell_type, key='rank_genes_groups')
current_df

In [ ]:
# Dot plot logfoldchanges
sc.pl.rank_genes_groups_dotplot(sdata, key='rank_genes_groups', n_genes=5, values_to_plot='logfoldchanges', min_logfoldchange=1, vmax=5, vmin=-5, cmap='bwr', dendrogram=False)

In [ ]:
# Dot plot mean expression (##)
sc.pl.rank_genes_groups_dotplot(sdata, key='rank_genes_groups_filtered', n_genes=5, dendrogram=False)

In [ ]:
# Dot plot mean expression (##)
sc.pl.dotplot(sdata, ['Slc17a7', 'Gad1', 'Gfap'], 'leiden', dendrogram=False)

In [ ]:
# Print markers 
markers = []
temp = pd.DataFrame(sdata.uns['rank_genes_groups']['names']).head(15)
for i in range(temp.shape[1]):
    curr_col = temp.iloc[:, i].to_list()
    markers = markers + curr_col
    # print(i, curr_col)
    print(i)
    for j in curr_col:
        print(j, end=' ')
    print('')

In [ ]:
# Plot UMAP with cluster labels w/ new color
fig, ax = plt.subplots(figsize=(10,7))
sc.pl.umap(sdata, color='leiden', legend_loc='on data',
           legend_fontsize=12, legend_fontoutline=2, frameon=False, 
           title='Top level clustering (leiden)', palette=cluster_pl, save=False, ax=ax)

In [ ]:
gene_list = ['Slc17a7', 'Gad1', 'Gad2', 'Sst', 'Pvalb', 'Npy', 'Vip', 'Pcp4', 'Cux2', 'Kif5a', 'Slc32a1', 'Nrgn', 'Sncg', 'Rorb', 'Tmsb4x']
# gene_list = ['Aqp4', 'Gfap', 'Plp1', 'Mbp', 'Mobp', 'Slc1a3', 'Pdgfra', 'Bsg', 'Vtn', 'Vim']

fig, axs = plt.subplots(nrows=3, ncols=5, figsize=(25, 12))
axs = axs.flatten()
for i, gene in enumerate(gene_list):
    ax = sc.pl.umap(sdata, color=gene, title=gene, ax=axs[i], show=False)
    
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
sns.scatterplot(x='column', y='row', hue='leiden', data=sdata.obs, palette=cluster_pl, s=2, legend=False)
ax.axis('off')
plt.show()

In [ ]:
# Plot single cluster
fig, ax = plt.subplots(figsize=(10, 10))
sns.scatterplot(x='column', y='row', hue='leiden', data=sdata.obs.loc[sdata.obs['leiden'] == '7', :], palette='tab20', s=2, legend=False, ax=ax)
ax.axis('off')
plt.show()

In [ ]:
from natsort import natsorted
for current_group in natsorted(sdata.obs['leiden'].unique()):
    
    # Plot single cluster
    fig, ax = plt.subplots(figsize=(10, 10))
    sns.scatterplot(x='column', y='row', hue='leiden', data=sdata.obs.loc[sdata.obs['leiden'] == current_group, :], palette=cluster_pl, s=2, legend=False, ax=ax)
    ax.title.set_text(current_group)
    ax.axis('off')
    plt.show()

### assign label

In [ ]:
# Plot UMAP with cluster labels w/ new color
fig, ax = plt.subplots(figsize=(10,7))
sc.pl.umap(sdata, color='leiden', legend_loc='on data',
           legend_fontsize=12, legend_fontoutline=2, frameon=False, 
           title=f'Level 2 {sub_id}', palette=cluster_pl, save=False, ax=ax)

In [ ]:
# Print markers 
markers = []
temp = pd.DataFrame(sdata.uns['rank_genes_groups_filtered']['names']).head(15)
for i in range(temp.shape[1]):
    curr_col = temp.iloc[:, i].to_list()
    markers = markers + curr_col
    # print(i, curr_col)
    print(i)
    for j in curr_col:
        print(j, end=' ')
    print('')

In [ ]:
# sc.pl.dotplot(sdata, ['Slc17a7', 'Gad1', 'Gad2'], 'leiden', dendrogram=False)
sc.pl.dotplot(sdata, ['Plp1', 'Gfap', 'Ctss', 'Bsg', 'Vtn'], 'leiden', dendrogram=False)

In [ ]:
# Change cluster label to cell type label
transfer_dict = {}

# Neuron
if sub_id == 'Neuron':
    level_2_list = [
        'Excitatory neuron', #0
        'Inhibitory neuron', #1
        'Excitatory neuron', #2
        'Excitatory neuron', #3
        'Inhibitory neuron', #4
        'Excitatory neuron', #5
        'Excitatory neuron', #6
        'Excitatory neuron', #7
        'Excitatory neuron', #8
        'Excitatory neuron', #9
        'Excitatory neuron', #10
        'Inhibitory neuron', #11
        'Excitatory neuron', #12
        'Excitatory neuron', #13
        'Excitatory neuron', #14
        'Excitatory neuron', #15
        'Inhibitory neuron', #16
        'Inhibitory neuron', #17
        'Inhibitory neuron', #18
        'Inhibitory neuron', #19
        'Excitatory neuron', #20
        'Inhibitory neuron', #21
        'Inhibitory neuron', #22
        'Excitatory neuron', #23
        'Excitatory neuron'] #24

# Glia
if sub_id == 'Glia':
    level_2_list = ['Oligodendrocyte 1', #0
                    'Oligodendrocyte 2', #1
                    'Astrocyte 1', #2
                    'Pericytes/Vascular endothelial cell 1', #3
                    'Astrocyte 2', #4
                    'Microglia', #5
                    'Oligodendrocyte 3', #6
                    'Oligodendrocyte 4', #7
                    'Vascular leptomeningeal cell 1', #8
                    'Pericytes/Vascular endothelial cell 2', #9
                    'Vascular leptomeningeal cell 2', #10
                    'Oligodendrocytes precursor cell', #11
                    'Vascular leptomeningeal cell 3', #12
                    'Astrocyte 3', #13 
                    'Ependymal cells', #14
                    'Unknown', #15
                    'Chorid plexus epithelial cells', #16
                    'Vascular leptomeningeal cell 4', #17
                   ]


for i in sorted(sdata.obs['leiden'].unique()):
    transfer_dict[i] = level_2_list[int(i)]

In [ ]:
# Print markers 
markers = []
temp = pd.DataFrame(sdata.uns['rank_genes_groups']['names']).head(10)
for i in range(temp.shape[1]):
    curr_col = temp.iloc[:, i].to_list()
    markers = markers + curr_col
    # print(i, curr_col)
    print(f"{i} - {level_2_list[i]}")
    for j in curr_col:
        print(j, end=',')
    print('')

In [ ]:
# Assign cell type to sdata
sdata.obs['level_2'] = sdata.obs['leiden'].values
sdata.obs = sdata.obs.replace({'level_2': transfer_dict})

# Remove NA 
# adata = adata[adata.obs['level_1'] != 'NA', :]

# Sort category
# level_2_order = ['Excitatory neuron', 'Inhibitory neuron']

# # # Sort category
level_2_order = natsorted(level_2_list)

sdata.obs['level_2'] = sdata.obs['level_2'].astype('category')
sdata.obs['level_2'].cat.reorder_categories(level_2_order, inplace=True)

In [ ]:
# Check color legend
level_2_pl = sns.color_palette("hls", len(level_2_order))
sns.palplot(level_2_pl, size=3)
plt.xticks(range(len(level_2_order)), level_2_order, size=10, rotation=45)
plt.tight_layout()
# plt.savefig(f'./figures/color_legend_top.png')
plt.show()

In [ ]:
# Save plots
# Plot UMAP with cluster labels w/ new color
sc.pl.umap(sdata, color='level_2', legend_loc='right margin',
           legend_fontsize=12, legend_fontoutline=2, frameon=False, 
           title=f'Level 2 {sub_id}', palette=level_2_pl, save=False)

fig, ax = plt.subplots(figsize=(10,10))
sc.pl.umap(sdata, color='level_2', legend_loc='on data',
           legend_fontsize=8, legend_fontoutline=1, frameon=False, ax=ax,
           title=f'Level 2 {sub_id}', palette=level_2_pl, save=False)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
sns.scatterplot(x='column', y='row', hue='level_2', data=sdata.obs, palette=level_2_pl, s=2, legend=False)
ax.axis('off')
plt.show()

In [ ]:
sc.pl.dotplot(sdata, ['Slc17a7', 'Gad1', 'Gad2'], 'level_2', dendrogram=False)

### add to adata

In [ ]:
# Map to original obj
if 'level_2' not in adata.obs.columns:
    adata.obs['level_2'] = adata.obs['level_1'].astype(object)
    
adata.obs['level_2'] = adata.obs['level_2'].astype(object)
adata.obs.loc[sdata.obs.index, 'level_2'] = sdata.obs['level_2'].values
adata.obs['level_2'].unique()

In [ ]:
# backup 
from datetime import datetime
date = datetime.today().strftime('%Y-%m-%d')
adata.write_h5ad(f"{out_path}/{date}-Hu-TissueRIBOmap-level2-bk.h5ad")

## Level_3 clustering

In [ ]:
# Load backup dataset 
adata = sc.read_h5ad(os.path.join(out_path, '2022-03-15-Hu-TissueRIBOmap-level2-bk.h5ad'))
adata

In [ ]:
# Subset
sub_id = 'Inhibitory neuron'
curr_cells = adata.obs['level_2'] == sub_id
sdata = adata[curr_cells, :]
sdata

In [ ]:
# Embedding parameters
emb_dict = {
    'Excitatory neuron': {'n_neighbors': 50, 'n_pcs': 15, 'min_dist': .1, 'cluster_resolution': 2},
    'Inhibitory neuron': {'n_neighbors': 50, 'n_pcs': 10, 'min_dist': .1, 'cluster_resolution': 1.5},
    # 'Oligodendrocyte': {'n_neighbors': 20, 'n_pcs': 5, 'min_dist': .1, 'cluster_resolution': 1},
    # 'Astrocyte': {'n_neighbors': 20, 'n_pcs': 5, 'min_dist': .1, 'cluster_resolution': 1},
}

save_embedding = False

In [ ]:
sub_level_fig_path = os.path.join(fig_path, sub_id)
if not os.path.exists(sub_level_fig_path):
    os.mkdir(sub_level_fig_path)

### clustering

In [ ]:
# Run PCA
sc.tl.pca(sdata, svd_solver='arpack', use_highly_variable=True)

# Plot explained variance 
sc.pl.pca_variance_ratio(sdata, log=False)

# Plot PCA
sc.pl.pca(sdata, color='protocol')

In [ ]:
%%time
# Computing the neighborhood graph
n_neighbors = emb_dict[sub_id]['n_neighbors']
n_pcs = emb_dict[sub_id]['n_pcs']
min_dist = emb_dict[sub_id]['min_dist']

sc.pp.neighbors(sdata, n_neighbors=n_neighbors, n_pcs=n_pcs, random_state=0)

# Run UMAP
sc.tl.umap(sdata, min_dist=min_dist, random_state=0)
sc.tl.diffmap(sdata, n_comps=n_pcs, random_state=0)

In [ ]:
%%time
# Run leiden cluster
cluster_resolution = emb_dict[sub_id]['cluster_resolution']
sc.tl.leiden(sdata, resolution = cluster_resolution)

# Plot UMAP with cluster labels 
sc.pl.umap(sdata, color='leiden')
sc.pl.diffmap(sdata, color='leiden')
n_clusters = sdata.obs['leiden'].unique().shape[0]

if save_embedding:
    # Save log
    with open(f'{fig_path}/log_{sub_id}.txt', 'w') as f:
        f.write(f"""Number of neighbor: {n_neighbors}
    Number of PC: {n_pcs}
    Resolution: {cluster_resolution}
    Min-distance: {min_dist}
    Number of clusters: {n_clusters}""")

    # save embeddings
    np.savetxt(f'{fig_path}/embedding_{sub_id}_umap.csv', sdata.obsm['X_umap'], delimiter=",")
    np.savetxt(f'{fig_path}/embedding_{sub_id}_diffmap.csv', sdata.obsm['X_diffmap'], delimiter=",")

In [ ]:
# Get colormap
cluster_pl = sns.color_palette("hls", n_clusters)
cluster_cmap = ListedColormap(cluster_pl.as_hex())
sns.palplot(cluster_pl)

In [ ]:
# Add log layer
sdata.layers['log_raw'] = np.log1p(sdata.layers['raw'])
sc.pp.normalize_total(sdata, layer='log_raw')

# Find gene markers for each cluster
sc.tl.rank_genes_groups(sdata, 'leiden', method='wilcoxon', layer='log_raw', pts=True, use_raw=False, n_genes=adata.shape[1])

# Filter markers
sc.tl.filter_rank_genes_groups(sdata, min_fold_change=.1, min_in_group_fraction=0.2, max_out_group_fraction=0.8)

In [ ]:
current_cell_type = '0'

current_df = sc.get.rank_genes_groups_df(sdata, group=current_cell_type, key='rank_genes_groups')
current_df

In [ ]:
# Dot plot logfoldchanges
sc.pl.rank_genes_groups_dotplot(sdata, key='rank_genes_groups', n_genes=5, values_to_plot='logfoldchanges', min_logfoldchange=1, vmax=5, vmin=-5, cmap='bwr', dendrogram=False)

In [ ]:
# Dot plot mean expression (##)
sc.pl.rank_genes_groups_dotplot(sdata, key='rank_genes_groups_filtered', n_genes=5, dendrogram=False)

In [ ]:
# Print markers 
markers = []
temp = pd.DataFrame(sdata.uns['rank_genes_groups']['names']).head(15)
for i in range(temp.shape[1]):
    curr_col = temp.iloc[:, i].to_list()
    markers = markers + curr_col
    # print(i, curr_col)
    print(i)
    for j in curr_col:
        print(j, end=' ')
    print('')

In [ ]:
# Plot UMAP with cluster labels w/ new color
fig, ax = plt.subplots(figsize=(10,7))
sc.pl.umap(sdata, color='leiden', legend_loc='on data',
           legend_fontsize=12, legend_fontoutline=2, frameon=False, 
           title='Top level clustering (leiden)', palette=cluster_pl, save=False, ax=ax)

In [ ]:
gene_list = ['Slc17a7', 'Gad1', 'Gad2', 'Sst', 'Pvalb', 'Npy', 'Vip', 'Pcp4', 'Cux2', 'Kif5a', 'Slc32a1', 'Nrgn', 'Sncg', 'Rorb', 'Tmsb4x']
# gene_list = ['Aqp4', 'Gfap', 'Plp1', 'Mbp', 'Mobp', 'Slc1a3', 'Pdgfra', 'Bsg', 'Vtn', 'Vim']

fig, axs = plt.subplots(nrows=3, ncols=5, figsize=(25, 12))
axs = axs.flatten()
for i, gene in enumerate(gene_list):
    ax = sc.pl.umap(sdata, color=gene, title=gene, ax=axs[i], show=False)
    
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
sns.scatterplot(x='column', y='row', hue='leiden', data=sdata.obs, palette=cluster_pl, s=2, legend=False)
ax.axis('off')
plt.show()

In [ ]:
from natsort import natsorted
for current_group in natsorted(sdata.obs['leiden'].unique()):
    
    # Plot single cluster
    fig, ax = plt.subplots(figsize=(10, 10))
    sns.scatterplot(x='column', y='row', data=adata.obs, color='#1111', s=2, legend=False, ax=ax)
    sns.scatterplot(x='column', y='row', hue='leiden', data=sdata.obs.loc[sdata.obs['leiden'] == current_group, :], palette=cluster_pl, s=4, legend=False, ax=ax)
    ax.title.set_text(current_group)
    ax.axis('off')
    plt.show()

In [ ]:
# Plot single cluster
fig, ax = plt.subplots(figsize=(10, 10))
sns.scatterplot(x='column', y='row', hue='leiden', data=sdata.obs.loc[sdata.obs['leiden'] == '14', :], palette='tab20', s=2, legend=False, ax=ax)
ax.axis('off')
plt.show()

### assign label

In [ ]:
# Plot UMAP with cluster labels w/ new color
fig, ax = plt.subplots(figsize=(10,7))
sc.pl.umap(sdata, color='leiden', legend_loc='on data',
           legend_fontsize=12, legend_fontoutline=2, frameon=False, 
           title=f'Level 3 {sub_id}', palette=cluster_pl, save=False, ax=ax)

In [ ]:
# Print markers 
markers = []
temp = pd.DataFrame(sdata.uns['rank_genes_groups_filtered']['names']).head(15)
for i in range(temp.shape[1]):
    curr_col = temp.iloc[:, i].to_list()
    markers = markers + curr_col
    # print(i, curr_col)
    print(i)
    for j in curr_col:
        print(j, end=' ')
    print('')

In [ ]:
# Change cluster label to cell type label
transfer_dict = {}

# Excitatory neuron
if sub_id == 'Excitatory neuron':
    level_3_list = ['Ex L2/3 1', #0
                    'Ex Mix 1', #1
                    'Ex TH 1', #2
                    'Ex DG', #3
                    'Ex Mix 2', #4
                    'Ex L6a 1', #5
                    'Ex TH 2', #6
                    'Ex Mix 3', #7
                    'Ex PIR', #8
                    'Ex L2/3 2', #9 
                    'Ex CTXsp', #10
                    'Ex L2/3 3', #11
                    'Ex L4/5 1', #12
                    'Ex L4', #13
                    'Ex Mix 4', #14
                    'Ex CA1', #15
                    'Ex MH', #16
                    'Ex L6', #17
                    'Ex MEA', #18
                    'Ex CA3', #19
                    'Ex TH 3', #20
                    'Ex Mix 5', #21
                   ]

# Inhibitory neuron
if sub_id == 'Inhibitory neuron':
    level_3_list = ['Inh LH/HY 1', #0
                    'Inh STR 1', #1
                    'Inh HY 1', #2
                    'Inh Pvalb 1', #3
                    'Inh LH/HY 2', #4
                    'Inh HY 2', #5
                    'Inh STR 2', #6
                    'Inh Mix 1', #7
                    'Inh HY 3', #8
                    'Inh Sst', #9 
                    'Inh Npy', #10
                    'Inh STR 3', #11
                    'Inh Pvalb 2', #12
                    'Inh STR 4', #13
                    'Inh Mix 2', #14
                    'Inh LH/HY 3', #15
                    'Inh STR 5', #16
                    'Inh HY 4', #17
                    'Inh HY 5', #18

                   ]


for i in sorted(sdata.obs['leiden'].unique()):
    transfer_dict[i] = level_3_list[int(i)]

In [ ]:
# Print markers 
markers = []
temp = pd.DataFrame(sdata.uns['rank_genes_groups']['names']).head(10)
for i in range(temp.shape[1]):
    curr_col = temp.iloc[:, i].to_list()
    markers = markers + curr_col
    # print(i, curr_col)
    print(f"{i} - {level_3_list[i]}")
    for j in curr_col:
        print(j, end=',')
    print('')

In [ ]:
# Assign cell type to sdata
sdata.obs['level_3'] = sdata.obs['leiden'].values
sdata.obs = sdata.obs.replace({'level_3': transfer_dict})

# Remove NA 
# adata = adata[adata.obs['level_1'] != 'NA', :]

# Sort category
level_3_order = natsorted(level_3_list)

# # Sort category
# level_2_order = ['Astrocyte', 'Oligodendrocyte', 'Oligodendrocytes precursor cell', 'Chorid plexus epithelial cell', 'Ependymal cell',
#                 'Pericytes/Vascular endothelial cell', 'Vascular leptomeningeal cell', 'Microglia', 'Unknown']

sdata.obs['level_3'] = sdata.obs['level_3'].astype('category')
sdata.obs['level_3'].cat.reorder_categories(level_3_order, inplace=True)

In [ ]:
# Check color legend
level_3_pl = sns.color_palette("hls", len(level_3_order))
sns.palplot(level_3_pl, size=3)
plt.xticks(range(len(level_3_order)), level_3_order, size=10, rotation=45)
plt.tight_layout()
# plt.savefig(f'./figures/color_legend_top.png')
plt.show()

In [ ]:
# Save plots
# Plot UMAP with cluster labels w/ new color
sc.pl.umap(sdata, color='level_3', legend_loc='right margin',
           legend_fontsize=12, legend_fontoutline=2, frameon=False, 
           title=f'Level 3 {sub_id}', palette=level_3_pl, save=False)

# sc.tl.rank_genes_groups(adata, 'level_1', method='wilcoxon')

# # Plot z-score heatmap
# sc.pl.rank_genes_groups_heatmap(adata, n_genes=5, groupby='top_level', min_logfoldchange=1, use_raw=False, swap_axes=True, 
#                                 vmin=-5, vmax=5, cmap='bwr', show_gene_labels=True,
#                                 dendrogram=False, figsize=(30, 15), save=False)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
sns.scatterplot(x='column', y='row', hue='level_3', data=sdata.obs, palette=level_3_pl, s=2, legend=False)
ax.axis('off')
plt.show()

### add to adata

In [ ]:
# Map to original obj
if 'level_3' not in adata.obs.columns:
    adata.obs['level_3'] = adata.obs['level_2'].astype(object)
    
adata.obs['level_3'] = adata.obs['level_3'].astype(object)
adata.obs.loc[sdata.obs.index, 'level_3'] = sdata.obs['level_3'].values
adata.obs['level_3'].unique()

In [ ]:
# backup 
from datetime import datetime
date = datetime.today().strftime('%Y-%m-%d')
adata.write_h5ad(f"{out_path}/{date}-Hu-TissueRIBOmap-level3-bk.h5ad")

## CA

In [ ]:
# Subset
sub_id = 'Ex CA3'
curr_cells = adata.obs['level_3'] == sub_id
sdata = adata[curr_cells, :]
sdata

In [ ]:
# Embedding parameters
emb_dict = {
    'Ex CA3': {'n_neighbors': 20, 'n_pcs': 30, 'min_dist': .1, 'cluster_resolution': .5},
}

save_embedding = False

In [ ]:
sub_level_fig_path = os.path.join(fig_path, sub_id)
if not os.path.exists(sub_level_fig_path):
    os.mkdir(sub_level_fig_path)

### clustering

In [ ]:
# Run PCA
sc.tl.pca(sdata, svd_solver='arpack', use_highly_variable=True)

# Plot explained variance 
sc.pl.pca_variance_ratio(sdata, log=False)

# Plot PCA
sc.pl.pca(sdata, color='protocol')

In [ ]:
%%time
# Computing the neighborhood graph
n_neighbors = emb_dict[sub_id]['n_neighbors']
n_pcs = emb_dict[sub_id]['n_pcs']
min_dist = emb_dict[sub_id]['min_dist']

sc.pp.neighbors(sdata, n_neighbors=n_neighbors, n_pcs=n_pcs, random_state=0)

# Run UMAP
sc.tl.umap(sdata, min_dist=min_dist, random_state=0)
sc.tl.diffmap(sdata, n_comps=n_pcs, random_state=0)

In [ ]:
%%time
# Run leiden cluster
cluster_resolution = emb_dict[sub_id]['cluster_resolution']
sc.tl.leiden(sdata, resolution = cluster_resolution)

# Plot UMAP with cluster labels 
sc.pl.umap(sdata, color='leiden')
sc.pl.diffmap(sdata, color='leiden')
n_clusters = sdata.obs['leiden'].unique().shape[0]

if save_embedding:
    # Save log
    with open(f'{fig_path}/log_{sub_id}.txt', 'w') as f:
        f.write(f"""Number of neighbor: {n_neighbors}
    Number of PC: {n_pcs}
    Resolution: {cluster_resolution}
    Min-distance: {min_dist}
    Number of clusters: {n_clusters}""")

    # save embeddings
    np.savetxt(f'{fig_path}/embedding_{sub_id}_umap.csv', sdata.obsm['X_umap'], delimiter=",")
    np.savetxt(f'{fig_path}/embedding_{sub_id}_diffmap.csv', sdata.obsm['X_diffmap'], delimiter=",")

In [ ]:
# Get colormap
cluster_pl = sns.color_palette("hls", n_clusters)
cluster_cmap = ListedColormap(cluster_pl.as_hex())
sns.palplot(cluster_pl)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
sns.scatterplot(x='column', y='row', hue='leiden', data=sdata.obs, palette=cluster_pl, s=2, legend=False)
ax.axis('off')
plt.show()

### assign label

In [ ]:
# Change cluster label to cell type label
transfer_dict = {}

level_3_list = ['Ex CA3', #0
                'Ex CA3', #1
                'Ex CA2', #2
                'Ex CA3', #3
                'Ex CA3', #4
               ]


for i in sorted(sdata.obs['leiden'].unique()):
    transfer_dict[i] = level_3_list[int(i)]

In [ ]:
# Assign cell type to sdata
sdata.obs['level_3'] = sdata.obs['leiden'].values
sdata.obs = sdata.obs.replace({'level_3': transfer_dict})

# Remove NA 
# adata = adata[adata.obs['level_1'] != 'NA', :]

# Sort category
level_3_order = ['Ex CA2', 'Ex CA3']

# # Sort category
# level_2_order = ['Astrocyte', 'Oligodendrocyte', 'Oligodendrocytes precursor cell', 'Chorid plexus epithelial cell', 'Ependymal cell',
#                 'Pericytes/Vascular endothelial cell', 'Vascular leptomeningeal cell', 'Microglia', 'Unknown']

sdata.obs['level_3'] = sdata.obs['level_3'].astype('category')
sdata.obs['level_3'].cat.reorder_categories(level_3_order, inplace=True)

### add to adata

In [ ]:
# Map to original obj
if 'level_3' not in adata.obs.columns:
    adata.obs['level_3'] = adata.obs['level_2'].astype(object)
    
adata.obs['level_3'] = adata.obs['level_3'].astype(object)
adata.obs.loc[sdata.obs.index, 'level_3'] = sdata.obs['level_3'].values
adata.obs['level_3'].unique()

In [ ]:
# backup 
from datetime import datetime
date = datetime.today().strftime('%Y-%m-%d')
adata.write_h5ad(f"{out_path}/{date}-Hu-TissueRIBOmap-level3.h5ad")

## Organize annotation

In [ ]:
adata.obs['level_1'].unique()

In [ ]:
adata.obs['level_2'].unique()

In [ ]:
adata.obs['level_3'].unique()

In [ ]:
for i in sorted(adata.obs['level_2'].unique()):
    print(i)

In [ ]:
# merge level 2 for glia

adata.obs['level_2'] = adata.obs['level_2'].astype(object)

oligo_list = ['Oligodendrocyte 1', 'Oligodendrocyte 2', 'Oligodendrocyte 3', 'Oligodendrocyte 4']
adata.obs.loc[adata.obs['level_2'].isin(oligo_list), 'level_2'] = 'Oligodendrocyte'

astro_list = ['Astrocyte 1', 'Astrocyte 2', 'Astrocyte 3']
adata.obs.loc[adata.obs['level_2'].isin(astro_list), 'level_2'] = 'Astrocyte'

vas_list = ['Chorid plexus epithelial cells', 'Ependymal cells', 'Pericytes/Vascular endothelial cell 1', 'Pericytes/Vascular endothelial cell 2', 
            'Vascular leptomeningeal cell 1', 'Vascular leptomeningeal cell 2', 'Vascular leptomeningeal cell 3', 'Vascular leptomeningeal cell 4']
adata.obs.loc[adata.obs['level_2'].isin(vas_list), 'level_2'] = 'Vascular cell'

adata.obs['level_2'] = adata.obs['level_2'].astype('category')
level_2_order = ['Excitatory neuron', 'Inhibitory neuron', 'Astrocyte', 'Oligodendrocyte', 'Oligodendrocytes precursor cell', 'Microglia', 'Vascular cell', 'Unknown']
adata.obs['level_2'] = adata.obs['level_2'].cat.reorder_categories(level_2_order)

In [ ]:
# backup 
from datetime import datetime
date = datetime.today().strftime('%Y-%m-%d')
adata.write_h5ad(f"{out_path}/{date}-Hu-TissueRIBOmap-level3.h5ad")

In [ ]:
# Get colormap
cluster_pl = sns.color_palette("hls", adata.obs['level_3'].nunique())
cluster_cmap = ListedColormap(cluster_pl.as_hex())
sns.palplot(cluster_pl)

In [ ]:
# Plot UMAP with cluster labels w/ new color
fig, ax = plt.subplots(figsize=(10,7))
sc.pl.umap(adata, color='level_3', legend_loc='right margin',
           legend_fontsize=12, legend_fontoutline=2, frameon=False, 
           title=f'Level 3', palette=cluster_pl, save=False, ax=ax)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
sns.scatterplot(x='column', y='row', hue='level_3', data=adata.obs, palette=cluster_pl, s=2, legend=False)
ax.axis('off')
plt.show()